# 📋 Multi-Agent Insurance Claim Orchestration

## Overview
The `orchestration.ipynb` notebook implements a **sophisticated multi-agent orchestration system** for insurance claim processing using **Microsoft Semantic Kernel** and **Azure AI Agent Service**. This notebook demonstrates advanced concurrent agent coordination to analyze insurance claims from multiple specialized perspectives simultaneously.

## 🏗️ Architecture & Components

### Core Technologies
- **Microsoft Semantic Kernel**: Agent orchestration framework
- **Azure AI Agent Service**: Cloud-based agent hosting and management
- **Concurrent Orchestration**: Parallel execution of multiple AI agents
- **Azure Cosmos DB Integration**: Real-time data access through custom plugins
- **Azure Identity**: Secure authentication for Azure services

### Agent Specializations
The system creates three specialized AI agents that work concurrently:

1. **🔍 Claim Reviewer Agent**
   - Validates claim documentation completeness
   - Analyzes damage assessments and cost estimates
   - Identifies inconsistencies or missing information
   - Provides VALID/QUESTIONABLE/INVALID determinations
   - Equipped with Cosmos DB plugin for data retrieval

2. **⚠️ Risk Analyzer Agent**
   - Detects fraud patterns and suspicious indicators
   - Assesses claim authenticity and credibility
   - Analyzes timing, amounts, and circumstances
   - Provides LOW/MEDIUM/HIGH risk assessments
   - Equipped with Cosmos DB plugin for historical analysis

3. **📋 Policy Checker Agent**
   - Validates coverage against policy terms
   - Interprets limits, deductibles, and exclusions
   - Handles multiple policy types (Auto, Commercial, Motorcycle, etc.)
   - Provides COVERED/NOT COVERED/PARTIAL COVERAGE determinations
   - Uses Azure AI Search for policy document analysis

## 🔧 Key Functions

### `create_specialized_agents()`
- Establishes Azure AI Agent client connections
- Configures each agent with specialized instructions and capabilities
- Sets up Cosmos DB plugins for data-enabled agents
- Configures polling options for optimal performance
- Returns a collection of ready-to-use specialized agents

### `run_insurance_claim_orchestration()`
- **Concurrent Processing**: All three agents analyze claims simultaneously
- **Intelligent Task Distribution**: Each agent receives specialized instructions
- **Real-time Data Access**: Agents can retrieve claim data using provided claim IDs
- **Comprehensive Reporting**: Generates unified analysis reports combining all agent outputs
- **Error Handling**: Robust exception management and resource cleanup
- **Progress Tracking**: Detailed logging of orchestration stages

## 🚀 Orchestration Flow

1. **Agent Creation**: Initializes three specialized insurance processing agents
2. **Concurrent Orchestration Setup**: Creates parallel execution framework
3. **Task Distribution**: Assigns specialized analysis tasks to each agent
4. **Parallel Execution**: All agents work simultaneously on their respective analyses
5. **Result Aggregation**: Collects and consolidates outputs from all agents
6. **Report Generation**: Creates comprehensive analysis report with all findings
7. **Resource Cleanup**: Properly terminates runtime and releases resources

## 📊 Output Format

The system generates a comprehensive **Insurance Claim Analysis Report** that includes:

- **Individual Agent Assessments**: Detailed analysis from each specialized perspective
- **Claim Validity Status**: Complete documentation and accuracy assessment
- **Risk Profile Analysis**: Fraud indicators and authenticity evaluation
- **Policy Coverage Determination**: Coverage eligibility and terms validation
- **Unified Recommendations**: Consolidated next steps based on all agent analyses

## 💡 Advanced Features

- **Concurrent Execution**: True parallelism for faster processing
- **Database Integration**: Real-time access to claim and policy data
- **Flexible Configuration**: Environment-based model and endpoint configuration
- **Comprehensive Logging**: Detailed progress tracking and status updates
- **Timeout Management**: 5-minute timeout for orchestration completion
- **Error Recovery**: Graceful handling of individual agent failures

## 🎯 Use Case Example

The notebook includes a practical example demonstrating the orchestration of claim analysis for "CL002", showing how the system processes claim details and retrieves additional data to provide comprehensive multi-agent analysis.

This implementation represents a cutting-edge approach to insurance claim processing, leveraging the power of concurrent AI agents to provide thorough, multi-perspective analysis that would traditionally require multiple human experts working in sequence.


In [ ]:
# Import necessary libraries for Semantic Kernel orchestration
import asyncio
import os
from typing import Dict, Any
from datetime import timedelta
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import (
    AzureAIAgent, 
    ConcurrentOrchestration
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Import the Cosmos DB plugin
from agents.tools import CosmosDBPlugin

async def create_specialized_agents():
    """Create our specialized insurance processing agents using Semantic Kernel."""
    
    print("🔧 Creating specialized insurance agents...")
    
    # Create Cosmos DB plugin instances for different agents
    cosmos_plugin_claims = CosmosDBPlugin()
    cosmos_plugin_risk = CosmosDBPlugin()
    
    # Get environment variables
    endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
    model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")
    
    agents = {}
    
    async with DefaultAzureCredential() as creds:
        client = AzureAIAgent.create_client(credential=creds, endpoint=endpoint)
        
        # Create Claim Reviewer Agent with Cosmos DB access
        print("🔍 Creating Claim Reviewer Agent...")
        claim_reviewer_definition = await client.agents.create_agent(
            model=model_deployment,
            name="ClaimReviewer",
            description="Expert Insurance Claim Reviewer Agent specialized in analyzing and validating insurance claims",
            instructions="""You are an expert Insurance Claim Reviewer Agent specialized in analyzing and validating insurance claims. Your primary responsibilities include:

**Core Functions:**
- Review claim documentation and assess completeness
- Analyze damage assessments and cost estimates
- Validate claim details against policy requirements
- Identify inconsistencies or missing information
- Provide detailed assessments with specific recommendations

**Analysis Guidelines:**
1. Use the Cosmos DB plugin to retrieve claim data by claim_id
2. Examine all claim details including dates, amounts, and descriptions
3. Assess the reasonableness of damage estimates
4. Check for proper documentation and evidence
5. Identify any red flags or unusual patterns

**Response Format:**
- Start with CLAIM STATUS: VALID/QUESTIONABLE/INVALID
- Provide detailed analysis of claim components
- List any missing documentation or concerns
- Give specific recommendations for next steps
- Be thorough but concise in your assessment

Use the available Cosmos DB functions to retrieve and analyze claim data."""
        )
        
        claim_reviewer_agent = AzureAIAgent(
            client=client,
            definition=claim_reviewer_definition,
            plugins=[cosmos_plugin_claims],
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000))
        )
        
        # Create Risk Analyzer Agent with Cosmos DB access
        print("⚠️ Creating Risk Analyzer Agent...")
        risk_analyzer_definition = await client.agents.create_agent(
            model=model_deployment,
            name="RiskAnalyzer",
            description="Expert Insurance Risk Analysis Agent specialized in detecting fraud patterns and assessing claim authenticity",
            instructions="""You are an expert Insurance Risk Analysis Agent specialized in detecting fraud patterns and assessing claim authenticity. Your primary responsibilities include:

**Core Functions:**
- Evaluate risk factors and fraud indicators
- Detect suspicious patterns in claim data
- Assess claim authenticity and credibility
- Analyze historical claim patterns
- Provide risk scores and recommendations

**Risk Assessment Guidelines:**
1. Use the Cosmos DB plugin to retrieve and analyze claim data
2. Look for unusual patterns, timing, or amounts
3. Check for consistency in claim details
4. Identify potential fraud indicators
5. Assess overall risk profile of the claim

**Fraud Indicators to Watch For:**
- Unusual timing of claims
- Inconsistent damage descriptions
- Suspicious amount patterns
- Multiple recent claims
- Geographic or temporal clustering

**Response Format:**
- Start with RISK LEVEL: LOW/MEDIUM/HIGH
- Provide detailed risk analysis
- List specific fraud indicators found (if any)
- Give risk score (1-10 scale)
- Recommend investigation actions if needed

Use the available Cosmos DB functions to thoroughly analyze claim data for risk assessment."""
        )
        
        risk_analyzer_agent = AzureAIAgent(
            client=client,
            definition=risk_analyzer_definition,
            plugins=[cosmos_plugin_risk],
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000))
        )
        
        # Create Policy Checker Agent (without Cosmos DB as it uses Azure AI Search)
        print("📋 Creating Policy Checker Agent...")
        policy_checker_definition = await client.agents.create_agent(
            model=model_deployment,
            name="PolicyChecker",
            description="Expert Insurance Policy Analysis Agent specialized in validating policy coverage and claim eligibility",
            instructions="""You are an expert Insurance Policy Analysis Agent specialized in validating policy coverage and claim eligibility. Your primary responsibilities include:

**Core Functions:**
- Analyze insurance policy terms and conditions
- Validate claim coverage under policy provisions
- Interpret policy limits, deductibles, and exclusions
- Determine eligibility for specific claim types
- Provide clear coverage determinations

**Policy Types You Handle:**
- Commercial Auto Policies
- Comprehensive Auto Policies  
- High Value Vehicle Policies
- Liability Only Policies
- Motorcycle Policies

**Analysis Guidelines:**
1. Review policy documents for relevant coverage
2. Check claim details against policy terms
3. Identify applicable limits, deductibles, and exclusions
4. Assess coverage eligibility
5. Provide clear determination with policy references

**Response Format:**
- Start with COVERAGE: COVERED/NOT COVERED/PARTIAL COVERAGE
- Reference specific policy sections
- Explain coverage limits and deductibles
- List any exclusions that apply
- Provide clear reasoning for determination

Provide professional, accurate, and thorough policy analysis."""
        )
        
        policy_checker_agent = AzureAIAgent(
            client=client,
            definition=policy_checker_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000))
        )
        
        agents = {
            'claim_reviewer': claim_reviewer_agent,
            'risk_analyzer': risk_analyzer_agent,
            'policy_checker': policy_checker_agent
        }
        
        print("✅ All specialized agents created successfully!")
        return agents, client

async def run_insurance_claim_orchestration(claim_details: str, claim_id: str = None):
    """Orchestrate multiple agents to process an insurance claim concurrently."""
    
    print(f"🚀 Starting Concurrent Insurance Claim Processing Orchestration")
    print(f"{'='*80}")
    
    # Create our specialized agents
    agents, client = await create_specialized_agents()
    
    # Create concurrent orchestration with all three agents
    orchestration = ConcurrentOrchestration(
        members=[agents['claim_reviewer'], agents['risk_analyzer'], agents['policy_checker']]
    )
    
    # Create and start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        print(f"\n🎯 Processing insurance claim with all agents working simultaneously")
        print(f"{'─'*60}")
        
        # Create specialized tasks for each agent
        base_task = f"""Analyze this insurance claim from your specialized perspective:

CLAIM DETAILS: {claim_details}"""
        
        if claim_id:
            base_task += f"\nCLAIM ID: {claim_id} (Use this to retrieve detailed claim data from Cosmos DB)"
        
        task = f"""{base_task}

AGENT-SPECIFIC INSTRUCTIONS:

Claim Reviewer Agent: 
- If claim_id is provided, use get_document_by_claim_id() to retrieve full claim details
- Review all claim documentation and assess completeness
- Validate damage estimates and repair costs
- Check for proper evidence and documentation
- Provide VALID/QUESTIONABLE/INVALID determination

Risk Analyzer Agent:
- If claim_id is provided, use get_document_by_claim_id() to retrieve claim data
- Analyze for fraud indicators and suspicious patterns
- Assess claim authenticity and credibility
- Check for unusual timing, amounts, or circumstances
- Provide LOW/MEDIUM/HIGH risk assessment

Policy Checker Agent:
- Analyze policy coverage for this type of claim
- Determine if claim is covered under policy terms
- Identify relevant exclusions, limits, or deductibles
- Provide COVERED/NOT COVERED/PARTIAL COVERAGE determination
- Reference specific policy sections

Each agent should provide comprehensive analysis from your area of expertise."""
        
        print(f"📋 Task assigned to all agents:")
        print(f"   • Claim Reviewer: Validate claim documentation and assess completeness")
        print(f"   • Risk Analyzer: Assess fraud risk and authenticity")
        print(f"   • Policy Checker: Determine coverage eligibility")
        
        # Invoke concurrent orchestration
        orchestration_result = await orchestration.invoke(
            task=task,
            runtime=runtime
        )
        
        # Get results from all agents
        results = await orchestration_result.get(timeout=300)  # 5 minute timeout
        
        print(f"\n🎉 All agents completed their analysis!")
        print(f"{'─'*60}")
        
        # Display individual results
        for i, result in enumerate(results, 1):
            agent_name = result.name if hasattr(result, 'name') else f"Agent {i}"
            content = str(result.content)
            print(f"\n🤖 {agent_name} Analysis:")
            print(f"{'─'*40}")
            print(content)
        
        # Create comprehensive analysis report
        comprehensive_analysis = f"""
# 🏢 Insurance Claim Analysis Report

## 📋 Claim Information
{claim_details}
{f"**Claim ID:** {claim_id}" if claim_id else ""}

## 🔍 Multi-Agent Concurrent Analysis Results

{chr(10).join([f"### {result.name} Assessment:{chr(10)}{chr(10)}{result.content}{chr(10)}" for result in results])}

## 📊 Summary and Recommendations

Based on the comprehensive concurrent analysis from our specialized AI agents:

1. **Claim Validity**: Assessed by our Claim Reviewer for documentation completeness and accuracy
2. **Risk Profile**: Evaluated by our Risk Analyzer for fraud indicators and authenticity
3. **Policy Coverage**: Determined by our Policy Checker for coverage eligibility and terms

**Next Steps**: This multi-agent analysis provides a thorough foundation for claim processing decisions. Review all agent assessments and follow their specific recommendations.

---
*Generated through concurrent multi-agent orchestration using Azure AI Agent Service and Semantic Kernel*
*Analysis completed simultaneously by specialized insurance processing agents*
"""
        
        print(f"\n✅ Concurrent Insurance Claim Orchestration Complete!")
        return comprehensive_analysis
        
    except Exception as e:
        print(f"❌ Error during orchestration: {str(e)}")
        raise
        
    finally:
        await runtime.stop_when_idle()
        print(f"\n🧹 Orchestration cleanup complete.")



In [ ]:
# Example usage:
claim_details = "Tell me your assessment for claim CL002"
claim_id = "CL002" 
result = await run_insurance_claim_orchestration(claim_details)
print(result)